# Project: Foundations of Computer Science, 2025-26

## Getting Environment Ready

In [1]:
! python -m pip install gdown pandas zstandard

## Preparing Dataset

In [2]:
import gdown
import zipfile

file_id = '1VuI1NnPzYlhHIMBy-2nBegFoQTATbf8K'
url = f"https://drive.google.com/uc?id={file_id}"
output = 'data.zip'
gdown.download(url, output, quiet=False)

with zipfile.ZipFile("data.zip", "r") as z:
    z.extractall('data')

Downloading...
From (original): https://drive.google.com/uc?id=1VuI1NnPzYlhHIMBy-2nBegFoQTATbf8K
From (redirected): https://drive.google.com/uc?id=1VuI1NnPzYlhHIMBy-2nBegFoQTATbf8K&confirm=t&uuid=5c9865e0-032a-4ab1-b6c2-96c38499da0d
To: /content/data.zip
100%|██████████| 57.2M/57.2M [00:02<00:00, 19.4MB/s]


## Project Text 

1. Create a single dataframe with the concatenation of all input csv files, adding a column called country
2. Extract all videos that have no tag.
3. For each channel, determine the total number of views
4. Save all rows with disabled comments and disabled ratings, or that have video_error_or_removed in a new dataframe called excluded, and remove those rows from the original dataframe.
5. Add a like_ratio column storing the ratio between the number of likes and of dislikes
6. Cluster the publish time into 10-minute intervals (e.g. from 02:20 to 02:30)
7. For each interval, determine the number of videos, average number of likes and of dislikes.
8. For each tag, determine the number of videos, Notice that tags contains a string with several tags.
9. Find the tags with the largest number of videos
10. For each (tag, country) pair, compute average ratio likes/dislikes
11. For each (trending_date, country) pair, the video with the largest number of views
12. Divide trending_date into three columns: year, month, day
13. For each (month, country) pair, the video with the largest number of views
14. Read all json files with the video categories
15. For each country, determine how many videos have a category that is not assignable.

1. Create a single dataframe with the concatenation of all input csv files, adding a column called country


In [2]:
import os
import pandas as pd
import zstandard as zstd

dataset_path = 'data/trendingYT'
CSVs = [x for x in os.listdir(dataset_path) if x.endswith('.zst')]

consolidated_df = pd.DataFrame()

for i in CSVs:
    country = i[:2]
    df = pd.read_csv(f"{dataset_path}/{i}", compression='zstd', encoding='ISO-8859-1')
    df['country'] = country
    consolidated_df = pd.concat([consolidated_df, df], ignore_index=True, axis=0, copy=False)

consolidated_df.tail()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country
375937,l884wKofd54,18.14.06,Enrique Iglesias - MOVE TO MIAMI (Official Vid...,EnriqueIglesiasVEVO,10,2018-05-09T07:00:01.000Z,"Enrique Iglesias feat. Pitbull|""MOVE TO MIAMI""...",25066952,268088,12783,9933,https://i.ytimg.com/vi/l884wKofd54/default.jpg,False,False,False,NEW SONG - MOVE TO MIAMI feat. Pitbull (Click ...,GB
375938,IP8k2xkhOdI,18.14.06,Jacob Sartorius - Up With It (Official Music V...,Jacob Sartorius,10,2018-05-11T17:09:16.000Z,"jacob sartorius|""jacob""|""up with it""|""jacob sa...",1492219,61998,13781,24330,https://i.ytimg.com/vi/IP8k2xkhOdI/default.jpg,False,False,False,THE OFFICIAL UP WITH IT MUSIC VIDEO!Get my new...,GB
375939,Il-an3K9pjg,18.14.06,Anne-Marie - 2002 [Official Video],Anne-Marie,10,2018-05-08T11:05:08.000Z,"anne|""marie""|""anne-marie""|""2002""|""two thousand...",29641412,394830,8892,19988,https://i.ytimg.com/vi/Il-an3K9pjg/default.jpg,False,False,False,Get 2002 by Anne-Marie HERE â¶ http://ad.gt/2...,GB
375940,-DRsfNObKIQ,18.14.06,Eleni Foureira - Fuego - Cyprus - LIVE - First...,Eurovision Song Contest,24,2018-05-08T20:32:32.000Z,"Eurovision Song Contest|""2018""|""Lisbon""|""Cypru...",14317515,151870,45875,26766,https://i.ytimg.com/vi/-DRsfNObKIQ/default.jpg,False,False,False,Eleni Foureira represented Cyprus at the first...,GB
375941,4YFo4bdMO8Q,18.14.06,KYLE - Ikuyo feat. 2 Chainz & Sophia Black [A...,SuperDuperKyle,10,2018-05-11T04:06:35.000Z,"Kyle|""SuperDuperKyle""|""Ikuyo""|""2 Chainz""|""Soph...",607552,18271,274,1423,https://i.ytimg.com/vi/4YFo4bdMO8Q/default.jpg,False,False,False,Debut album 'Light of Mine' out now: http://ky...,GB


2. Extract all videos that have no tag.

In [3]:
consolidated_df[consolidated_df['tags'].isnull()].head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country


3. For each channel, determine the total number of views

In [4]:
view_counts = (
    consolidated_df[['channel_title', 'views']]
    .groupby(['channel_title'])
    .sum(['views']))

view_counts.head(10)

,views
channel_title,
! ì¸ìì ë¬´ì¨ì¼ì´,3942977
!!8æã ãé¢ç½ãã¿å¤§éå,50207
!BTSã»TWICE ã¾ã¨ã,7310
!Los amorosos ViralesÂ¡,6069
!t Live,240038
!å ã©ãã®ä¸ãèµ°ã,11528
!å³¶ã¡ãã,1061020
!è¡æçãªåç»,9112
# #,2318


4. Save all rows with disabled comments and disabled ratings, or that have video_error_or_removed in a new dataframe called excluded, and remove those rows from the original dataframe.

In [5]:
excluded = consolidated_df[
    ((consolidated_df['comments_disabled'] == True) & (consolidated_df['ratings_disabled'] == True))
    | (consolidated_df['video_error_or_removed'] == True)
]

consolidated_df = consolidated_df.drop(excluded.index)

excluded.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country
1435,Kn5UgGQukYQ,17.21.11,Breaking Bad's Bryan Cranston on Meeting Charl...,hudsonunionsociety,1,2016-10-14T21:14:51.000Z,"Breaking Bad|""Bryan Cranston""|""malcom in the m...",15058,0,0,0,https://i.ytimg.com/vi/Kn5UgGQukYQ/default.jpg,True,True,False,The Hudson Union www.hudsonunionsociety.com is...,US
1667,Kn5UgGQukYQ,17.22.11,Breaking Bad's Bryan Cranston on Meeting Charl...,hudsonunionsociety,1,2016-10-14T21:14:51.000Z,"Breaking Bad|""Bryan Cranston""|""malcom in the m...",34207,0,0,0,https://i.ytimg.com/vi/Kn5UgGQukYQ/default.jpg,True,True,False,The Hudson Union www.hudsonunionsociety.com is...,US
1914,Kn5UgGQukYQ,17.23.11,Breaking Bad's Bryan Cranston on Meeting Charl...,hudsonunionsociety,1,2016-10-14T21:14:51.000Z,"Breaking Bad|""Bryan Cranston""|""malcom in the m...",36137,0,0,0,https://i.ytimg.com/vi/Kn5UgGQukYQ/default.jpg,True,True,False,The Hudson Union www.hudsonunionsociety.com is...,US
2119,Kn5UgGQukYQ,17.24.11,Breaking Bad's Bryan Cranston on Meeting Charl...,hudsonunionsociety,1,2016-10-14T21:14:51.000Z,"Breaking Bad|""Bryan Cranston""|""malcom in the m...",36579,0,0,0,https://i.ytimg.com/vi/Kn5UgGQukYQ/default.jpg,True,True,False,The Hudson Union www.hudsonunionsociety.com is...,US
2203,RK_B4Ez4_5Q,17.25.11,Verizon 360 Live: The Macyâs Thanksgiving Da...,Verizon,24,2017-11-23T19:51:07.000Z,"live stream|""360 video""|""fun videos for kids""|...",2618344,45197,2315,3332,https://i.ytimg.com/vi/RK_B4Ez4_5Q/default.jpg,False,False,True,"This year, we hid special offers throughout th...",US


5. Add a like_ratio column storing the ratio between the number of likes and of dislikes

In [6]:
consolidated_df['like_ratio'] = consolidated_df['likes'] / consolidated_df['dislikes']

consolidated_df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country,like_ratio
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...,US,19.395482
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John...",US,15.812724
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO â¶ \n\nSUBSCRIBE âº ...,US,27.352126
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...,US,15.273273
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,US,66.483157


6. Cluster the publish time into 10-minute intervals (e.g. from 02:20 to 02:30)

In [7]:
consolidated_df['publish_time'] = pd.to_datetime(consolidated_df['publish_time'], errors='coerce').dt.round('10T')

consolidated_df.head()

/var/folders/c0/6cp8blwj6tv35_5pwvy16mp80000gn/T/ipykernel_22595/1819962047.py:1: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  consolidated_df['publish_time'] = pd.to_datetime(consolidated_df['publish_time'], errors='coerce').dt.round('10T')


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country,like_ratio
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13 17:10:00+00:00,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...,US,19.395482
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13 07:30:00+00:00,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John...",US,15.812724
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12 19:10:00+00:00,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO â¶ \n\nSUBSCRIBE âº ...,US,27.352126
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13 11:00:00+00:00,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...,US,15.273273
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12 18:00:00+00:00,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,US,66.483157


7. For each interval, determine the number of videos, average number of likes and of dislikes.

In [8]:
interval_stats = (
    consolidated_df[['publish_time', 'views', 'likes', 'dislikes', 'comment_count']]
    .groupby(['publish_time'])
    .agg({'likes': ['mean'], 'dislikes': ['mean'], 'publish_time': ['count']})
)
interval_stats.head()

,likes,dislikes,publish_time
,mean,mean,count
publish_time,,,
2006-07-23 08:20:00+00:00,459.000000,152.0000,1
2007-03-05 16:20:00+00:00,336.666667,2.0000,9
2007-06-25 06:50:00+00:00,579.833333,11.5000,12
2007-12-03 20:50:00+00:00,187.937500,15.6875,16
2008-01-07 21:20:00+00:00,99.900000,2.0000,10


8. For each tag, determine the number of videos, Notice that tags contains a string with several tags.

In [9]:
consolidated_df['isolated_tags'] = consolidated_df['tags'].str.split('|').head()
tags_video_df = consolidated_df.explode('isolated_tags')

tags_video_df.groupby(['isolated_tags']).agg({'video_id': ['count']})

,video_id
,count
isolated_tags,
"Lele Pons""",1
"""Getting My Driver's License",1
"""Season 12""",1
"""alesso""",1
"""anitta""",1
...,...
SHANtell martin,1
last week tonight trump presidency,1


9. Find the tags with the largest number of videos

In [10]:
tags_video_df.groupby(['isolated_tags']).agg({'video_id': ['count']}).sort_values(by=('video_id', 'count'), ascending=False).head(1)

,video_id
,count
isolated_tags,
"""funny""",2


10. For each (tag, country) pair, compute average ratio likes/dislikes

In [11]:
tags_video_df.groupby(['isolated_tags', 'country']).agg({'like_ratio': ['mean']})

,,like_ratio
,,mean
isolated_tags,country,
"Lele Pons""",US,27.352126
"""Getting My Driver's License",US,27.352126
"""Season 12""",US,15.273273
"""alesso""",US,27.352126
"""anitta""",US,27.352126
...,...,...
SHANtell martin,US,19.395482
last week tonight trump presidency,US,15.812724


11. For each (trending_date, country) pair, the video with the largest number of views

In [12]:
consolidated_df.groupby(['trending_date', 'country']).agg({'views': ['max']})

views
                             max
trending_date country           
17.01.12      CA        56367282
              DE        56367282
              FR         7281189
              GB       143408235
              IN        56367282
...                          ...
18.31.05      JP         3057987
              KR         4150448
              MX        20723565
              RU         3125598
              US       217750076

[1967 rows x 1 columns]

12. Divide trending_date into three columns: year, month, day

In [13]:
consolidated_df['year'] = consolidated_df['trending_date'].str[:2].astype(int) + 2000
consolidated_df['day'] = consolidated_df['trending_date'].str[3:-3].astype(int)
consolidated_df['month'] = consolidated_df['trending_date'].str[-2:].astype(int)

consolidated_df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,...,comments_disabled,ratings_disabled,video_error_or_removed,description,country,like_ratio,isolated_tags,year,day,month
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13 17:10:00+00:00,SHANtell martin,748374,57527,2966,...,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...,US,19.395482,[SHANtell martin],2017,14,11
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13 07:30:00+00:00,"last week tonight trump presidency|""last week ...",2418783,97185,6146,...,False,False,False,"One year after the presidential election, John...",US,15.812724,"[last week tonight trump presidency, ""last wee...",2017,14,11
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12 19:10:00+00:00,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,...,False,False,False,WATCH MY PREVIOUS VIDEO â¶ \n\nSUBSCRIBE âº ...,US,27.352126,"[racist superman, ""rudy"", ""mancuso"", ""king"", ""...",2017,14,11
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13 11:00:00+00:00,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,...,False,False,False,Today we find out if Link is a Nickelback amat...,US,15.273273,"[rhett and link, ""gmm"", ""good mythical morning...",2017,14,11
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12 18:00:00+00:00,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,...,False,False,False,I know it's been a while since we did this sho...,US,66.483157,"[ryan, ""higa"", ""higatv"", ""nigahiga"", ""i dare y...",2017,14,11


13. For each (month, country) pair, the video with the largest number of views

In [14]:
consolidated_df.groupby(['country', 'month']).agg({'views': ['max']})

views
                     max
country month           
CA      1       43067983
        2       47362934
        3       61163906
        4       53071887
        5       98938809
...                  ...
US      4       74523616
        5      217750076
        6      225211923
        11     102012605
        12     149376127

[77 rows x 1 columns]

14. Read all json files with the video categories

In [24]:
from pathlib import Path
import os
import pandas as pd
import json

dataset_path = Path('data/trendingYT')
JSONs = [x for x in os.listdir(dataset_path) if x.endswith('.json')]

category_rows = []
for i in JSONs:
    country = i[:2]
    blob = json.loads((dataset_path / i).read_text(encoding='ISO-8859-1'))['items']
    for item in blob:
        cid = item.get('id')
        snippet = item.get('snippet', {})
        title = snippet.get('title')
        assignable = snippet.get('assignable')
        category_rows.append({
            'country': country,
            'category_id': str(cid),
            'category_title': title,
            'assignable': assignable,
        })

category_df = pd.DataFrame(category_rows)
category_df['category_id'] = category_df['category_id'].astype(str)

print(category_df.head())

  country category_id    category_title  assignable
0      IN           1  Film & Animation        True
1      IN           2  Autos & Vehicles        True
2      IN          10             Music        True
3      IN          15    Pets & Animals        True
4      IN          17            Sports        True


15. For each country, determine how many videos have a category that is not assignable.

In [ ]:
# 15. For each country, determine how many videos have a category that is not assignable.

consolidated_df['category_id'] = consolidated_df['category_id'].astype(str)

master_df = consolidated_df.merge(category_df[['country','category_id','assignable']], on=['country','category_id'], how='left')

not_assignable = master_df[master_df['assignable'] == False]

# where no mapping exists so it's a Nan value
missing_mapping = master_df[master_df['assignable'].isna()]

counts_not_assignable = not_assignable.groupby('country').size().rename('not_assignable_count')
counts_missing = missing_mapping.groupby('country').size().rename('missing_category_mapping')

summary = pd.concat([counts_not_assignable, counts_missing], axis=1).fillna(0).astype(int)
print('\nPer-country summary (not assignable / missing mapping):')
print(summary)


Per-country summary (not assignable / missing mapping):
         not_assignable_count  missing_category_mapping
country                                                
CA                        130                        74
DE                        110                       242
FR                        112                       111
GB                         20                        90
IN                        221                        74
KR                        167                       286
MX                          3                       251
RU                        195                      1538
US                         57                         0
JP                          0                        18
